# Network modules in Pytorch

In [1]:
import torch
import torch.nn as nn
import numpy as np

## Parameter class

In [2]:
learn_data = nn.Parameter(torch.randn(10, 10))
print(type(learn_data), learn_data.requires_grad)
print(learn_data.shape)
print(type(learn_data.data))


<class 'torch.nn.parameter.Parameter'> True
torch.Size([10, 10])
<class 'torch.Tensor'>


## Module class

### Get number of parameters in Module

In [3]:
def get_num_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [2]:
class Dummy(nn.Module):
    def __init__(self):
        super(Dummy, self).__init__()
        
        self.weight = nn.Parameter(torch.randn(10, 10))
        gamma = torch.randn(3, 10)
        self.conv = nn.Conv2d(3, 10, 5)
        layers = [self.conv, self.conv]

        self.style = nn.Sequential(*layers)
    
    def forward(self, x):
        return x * self.weight
    
net = Dummy()
params = net.parameters()

for i in params:
    print(i.shape)

print(net)
print(net.state_dict().keys())

print("The total number of parameters in module is:")
pytorch_total_params = sum(p.numel() for p in net.parameters())
print(pytorch_total_params)

AttributeError: 'Dummy' object has no attribute 'device'

## BatchNorm

In [8]:
norm = nn.BatchNorm2d(512)

input = torch.randn(1, 512, 64, 64)
output = norm(input)
print(output.shape)

print("The total number of parameters in module is:")
pytorch_total_params = get_num_parameters(norm)
print(pytorch_total_params)

torch.Size([1, 512, 64, 64])
The total number of parameters in module is:
1024


## Linear layer

In [26]:
from icecream import ic as print

fc = nn.Linear(8, 128)

input_data = torch.randn(64, 60, 8, requires_grad=True)
input_data = torch.randn(64, 60, 8)
# by default, the requires_grad of parameters is True
output = fc(input_data)
print(input_data.requires_grad, output.requires_grad) 
print(fc.weight.requires_grad, fc.bias.requires_grad)

print("Use torch.no_grad")
with torch.no_grad():
    output = fc(input_data)
    print(fc.weight.requires_grad, fc.bias.requires_grad)
    print(input_data.requires_grad, output.requires_grad)

print("Set network requires_grad False")
input_data = torch.randn(64, 60, 8, requires_grad=True)
input_data = torch.randn(64, 60, 8)
fc.requires_grad_(False)
print(fc.weight.requires_grad, fc.bias.requires_grad)
output = fc(input_data)
print(input_data.requires_grad, output.requires_grad)

ic| input_data.requires_grad: False, output.requires_grad: True
ic| fc.weight.requires_grad: True, fc.bias.requires_grad: True
ic| 'Use torch.no_grad'
ic| fc.weight.requires_grad: True, fc.bias.requires_grad: True
ic| input_data.requires_grad: False, output.requires_grad: False
ic| 'Set network requires_grad False'
ic| fc.weight.requires_grad: False, fc.bias.requires_grad: False
ic| input_data.requires_grad: False, output.requires_grad: False


(False, False)

## Identity Module

In [3]:
## To be as a placeholder in code, don't use any passed arguments
m = nn.Identity(54, unused_argument1=0.1, unused_argument2=False)
input = torch.randn(3, 4)
print(input)

output = m(input)  # Do nothing
print(output)

tensor([[-0.8822, -0.3836, -0.4453,  0.4732],
        [-0.2166,  1.7199,  0.8660, -1.1443],
        [-1.3421,  0.9498, -0.0602, -0.1829]])
tensor([[-0.8822, -0.3836, -0.4453,  0.4732],
        [-0.2166,  1.7199,  0.8660, -1.1443],
        [-1.3421,  0.9498, -0.0602, -0.1829]])


## Conv2d, Conv3d

In [43]:
### 反卷积 https://pytorch.org/docs/stable/generated/torch.nn.ConvTranspose2d.html
import torch
import torch.nn as nn

## Common convolutional layers
conv = nn.Conv2d(3, 5, 3, 2, 1)
print(conv.weight.shape)
print(conv.bias.shape)
print(conv.bias)
conv.weight.data.normal_()
conv.bias.data.zero_()
print(conv.bias)

weight = getattr(conv, "weight")
print(weight.shape)
print(weight.data.size(1), weight.data[0][0])

## Apply convolution
print("\nApply convolution")
input = torch.randn(2, 3, 10, 10)
output = conv(input)
print(output.shape)

torch.Size([5, 3, 3, 3])
torch.Size([5])
Parameter containing:
tensor([-0.1532, -0.0904,  0.1121,  0.1455,  0.0122], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
torch.Size([5, 3, 3, 3])
3 tensor([[ 0.4743,  0.3974, -1.2316],
        [-2.2109, -1.5763, -1.4718],
        [ 0.5063,  1.5260,  0.7495]])

Apply convolution
torch.Size([2, 5, 5, 5])


## Conv1x1

In [11]:
from icecream import ic as print

conv1 = nn.Conv2d(1024, 64, 1, stride=1, padding=0)

input = torch.rand(2, 1024, 1, 1)

output = conv1(input)
print(output.shape)

num_param_conv1 = get_num_parameters(conv1)
print(num_param_conv1)

fc = nn.Linear(1024, 64)
output = fc(input.squeeze())
print(output.shape)

num_param_fc = get_num_parameters(fc)
print(num_param_fc)


ic| output.shape: torch.Size([2, 64, 1, 1])
ic| num_param_conv1: 65600
ic| output.shape: torch.Size([2, 64])
ic| num_param_fc: 65600


65600

## Embedding

In [15]:
import torch
import torch.nn as nn

image_emb = nn.Embedding(5, 768)
print(image_emb.weight.shape)

# input = torch.LongTensor(4,)
tok = torch.tensor([0], dtype=torch.long)
input = tok.repeat(4, 10)
print(input.shape)
output = image_emb(input)
print(output.shape, output.min(), output.max())


torch.Size([5, 768])
torch.Size([4, 10])
torch.Size([4, 10, 768]) tensor(-3.5268, grad_fn=<MinBackward1>) tensor(2.7718, grad_fn=<MaxBackward1>)


## Compute the Convolution output tensor size

In [11]:
def Cov3DSize(input, cin, cout, kernel_size, stride, padding):
    conv = nn.Conv3d(cin, cout, kernel_size=kernel_size, stride=stride, padding=padding)
    output = conv(input)
    print(output.shape)
    return output

def MaxPool3D(input, kernel_size, stride):
    pool = nn.MaxPool3d(kernel_size=kernel_size, stride=stride)
    output = pool(input)
    print("MaxPool3D shape: ", output.shape)
    return output

input = torch.rand(20, 3, 5, 224, 224)
input = Cov3DSize(input, 3, 96, kernel_size=(5,7,7), stride=(1,2,2), padding=0)
input = MaxPool3D(input, kernel_size=(1,3,3), stride=(1,2,2))




torch.Size([20, 96, 1, 109, 109])
MaxPool3D shape:  torch.Size([20, 96, 1, 54, 54])


tensor([[[[[ 0.5023,  0.5023,  0.4128,  ...,  0.4558,  0.3107,  0.4717],
           [ 0.5023,  0.5793,  0.3253,  ...,  0.4558,  0.3143,  0.4888],
           [ 0.3287,  0.2374,  0.5073,  ...,  0.3702,  0.3143,  0.4888],
           ...,
           [ 0.3105,  0.3293,  0.4435,  ...,  0.2628,  0.4014,  0.4014],
           [ 0.2706,  0.3995,  0.4435,  ...,  0.3990,  0.2933,  0.1918],
           [ 0.4065,  0.3995,  0.3467,  ...,  0.2890,  0.1323,  0.1790]]],


         [[[ 0.2466,  0.2063,  0.1703,  ...,  0.0763, -0.0086, -0.0442],
           [-0.0941,  0.2566,  0.1294,  ...,  0.0763, -0.1647, -0.1810],
           [ 0.0423,  0.0423,  0.0011,  ...,  0.1201,  0.1201,  0.0577],
           ...,
           [ 0.0726,  0.0833,  0.2228,  ..., -0.0621,  0.2760,  0.0202],
           [ 0.0406,  0.0515,  0.2228,  ..., -0.0695,  0.2760, -0.0345],
           [ 0.0502,  0.0502, -0.0586,  ...,  0.0078, -0.0091,  0.1370]]],


         [[[-0.0618, -0.0442, -0.0197,  ...,  0.1594, -0.0027,  0.0232],
           

## MaxPool2d, MaxPool3d
[https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html)

tensor([[[[ 1.2346e+00,  3.7939e-01, -1.9367e+00,  1.3094e+00,  4.6663e-01],
          [ 3.4819e-01, -5.6895e-01, -1.5108e+00,  7.0780e-01,  9.0777e-01],
          [ 4.6561e-01,  2.1563e+00, -2.3255e+00, -6.6472e-01,  5.5147e-01],
          [ 9.4951e-01, -3.3740e-01, -6.2161e-02,  4.8511e-01, -1.4107e-01],
          [ 6.3537e-01,  1.5464e+00,  2.0419e+00,  2.4523e-01,  2.2240e+00]],

         [[-5.4798e-01,  1.6411e-01, -2.4302e-01, -2.1069e+00, -1.1748e+00],
          [ 5.8614e-01, -1.0792e+00,  8.6184e-02,  2.6984e-02,  3.7680e-01],
          [-4.2325e-02,  5.9603e-01, -5.9892e-01,  6.5933e-01,  9.7178e-01],
          [-1.4920e+00, -2.1150e-01, -2.5007e-01, -2.6350e-01, -7.3031e-01],
          [-8.8017e-01, -1.0805e+00, -8.5367e-01,  2.3745e+00,  1.7426e+00]],

         [[ 3.5111e-01, -1.0492e+00, -6.6652e-01,  1.4015e+00,  2.0608e+00],
          [-1.2575e+00, -4.0794e-01,  2.2955e-02, -1.0286e+00,  6.4215e-01],
          [-8.6350e-01, -2.7992e-01, -2.8991e-01,  9.1334e-01,  3.0354e-

### AdaptiveAvgPool2d

In [4]:
adp_pool = torch.nn.AdaptiveAvgPool2d((256, 256))
input = torch.randn(1, 3, 512, 512)
output = adp_pool(input)
print(output.shape)

pytorch_total_params = sum(p.numel() for p in adp_pool.parameters())
print(pytorch_total_params)

torch.Size([1, 3, 256, 256])
0


In [53]:
upsample = nn.Upsample(scale_factor=2, mode='nearest')
print([i for i in upsample.parameters()])

print("The total number of parameters in module is:")
pytorch_total_params = sum(p.numel() for p in upsample.parameters())
print(pytorch_total_params)

[]
The total number of parameters in module is:
0


In [56]:
style = torch.randn(1, 4, 2, 2)
gamma, beta = style.chunk(2, 1)
print(gamma.shape, beta.shape)
print(style)
print(gamma)
print(beta)

torch.Size([1, 2, 2, 2]) torch.Size([1, 2, 2, 2])
tensor([[[[ 0.3909, -0.1451],
          [-1.6407, -1.0667]],

         [[-0.2172,  0.2076],
          [ 1.9867,  0.7596]],

         [[ 1.0905, -0.9003],
          [ 0.4875,  0.6588]],

         [[ 1.1397,  1.1139],
          [ 0.7776,  1.0089]]]])
tensor([[[[ 0.3909, -0.1451],
          [-1.6407, -1.0667]],

         [[-0.2172,  0.2076],
          [ 1.9867,  0.7596]]]])
tensor([[[[ 1.0905, -0.9003],
          [ 0.4875,  0.6588]],

         [[ 1.1397,  1.1139],
          [ 0.7776,  1.0089]]]])


## Misc math operations

## Conca, Stack operations

torch.Size([6, 5, 96, 96])
torch.Size([5, 2, 1])


a的shape： torch.Size([2, 2, 3])
b: (tensor([[[1, 2, 3],
         [4, 5, 6]]]), tensor([[[ 7,  8,  9],
         [10, 11, 12]]])) 2
c: (tensor([[[1, 2, 3]],

        [[7, 8, 9]]]), tensor([[[ 4,  5,  6]],

        [[10, 11, 12]]]))
